# Importing Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Loading the Datasets

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='z0u1vzt8qo_0xobAJTt1SWEn6yZmKSWG7oH3mQR1KjUF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'universityadmissionprediction-donotdelete-pr-d6ymzrsmcxijbu'
object_key = 'Admission_Predict.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()
data

# Analysing the datas

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
data.skew()

In [ ]:
data.info()

# Handling Missing Values from the DataSets

In [ ]:
data.isnull() #method returns datafram object where all values replaced with boolean val true or false

In [ ]:
data.notnull()

In [ ]:
data.sum()

In [ ]:
data.isna().sum() 

# Data Visualization:

# 1.Univariate Analysis

# 2.Bivariate Analysis

# 3.Multi-Variate Analysis

# Univariate Analysis

In [ ]:
sns.displot(x=data["GRE Score"], kde=True, color='Red')
plt.title("GRE score distribution of data");

In [ ]:
sns.displot(x=data["TOEFL Score"], kde=True, color='Black')
plt.title("TOEFL score distribution of data");

In [ ]:
sns.countplot(x=data["University Rating"]);
plt.title("Universities Star Rating by applicants");

In [ ]:
sns.displot(x=data["SOP"], kde=True, color='yellow');
plt.title("Distribution for ratings on SOP");

In [ ]:
sns.displot(x=data["CGPA"], kde=True, color='Darkgreen');
plt.title("Distribution of CGPA Score");

In [ ]:
sns.countplot(x=data["Research"]);
plt.title("Research Count");

# Bivariate Analysis

In [ ]:
cols = data.columns
features = [i for i in data.columns if i != 'Chance of Admit']
label = 'Chance of Admit'
features

In [ ]:
sns.regplot(data['SOP'],data['CGPA'],color='Darkblue')


In [ ]:
sns.boxplot(data['SOP'],data['CGPA'],color='Darkblue')

# Multi-Variate Analysis

In [ ]:
sns.pairplot(data,hue='Research')

In [ ]:
corr_matrix = data.corr()
plt.figure(figsize = (15, 15))
sns.heatmap(corr_matrix,annot=True,fmt='0.2f')
plt.title("Correlation Matrix", fontsize = 20)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(data.loc[:,"GRE Score":"Research"],diagonal="kde",figsize=(20,15))
plt.show

# Split the data into dependent and independent variables

In [ ]:
x = data.iloc[:,0:7]
x

In [ ]:
x.head()

In [ ]:
y = data.iloc[:,8:9 ]
y

In [ ]:
y.head()

In [ ]:
print(f'x contains: {x.shape[0]} rows and {x.shape[1]} columns')

In [ ]:
print(f'y contains: {y.shape}')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x[x.columns] = scaler.fit_transform(x[x.columns])

In [ ]:
x.head()

# Splitting The Data Into Train And Test

In [ ]:
X=data.drop(['Chance of Admit '],axis=1) #input data_set
y=data['Chance of Admit '] #output labels

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

# MODELING AND TRAINING


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)


In [ ]:
y_predict=model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import numpy as np
print('r2 score:',r2_score(y_test,y_predict))
print('Mean Absolute Error:', mean_absolute_error(y_test, y_predict))  
print('Mean Squared Error:', mean_squared_error(y_test, y_predict))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_predict)))

In [ ]:
y_train = (y_train>0.5)
y_test = (y_test>0.5)

In [ ]:
from sklearn.linear_model._logistic import LogisticRegression

lore = LogisticRegression(random_state=0, max_iter=1000)

lr = lore.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix

print('Accuracy Score:', accuracy_score(y_test, y_pred))  
print('Recall Score:', recall_score(y_test, y_pred))  
print('ROC AUC Score:', roc_auc_score(y_test, y_pred))
print('Confussion Matrix:\n', confusion_matrix(y_test, y_pred))


# SAVING THE MODEL

In [ ]:
import pickle

In [ ]:
pickle.dump(lr, open("university.pkl", "wb")) #logistic regression model

# HOSTING THE MODEL

In [ ]:
import pickle

In [ ]:
lr = pickle.load(open("university.pkl", "rb")) #logistic regression model

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient
import json

# Authenticate and set space

In [ ]:
wml_credentials = {
    "apikey":"d2-3twnS_oRmh5yFAxs0aK7kpqJcN3_1qQy6IX3-P413",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [ ]:
wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.spaces.list()

In [ ]:
SPACE_ID="77330267-b27a-47ac-8c71-e7808afcd79e"

In [ ]:
wml_client.set.default_space(SPACE_ID)

In [ ]:
wml_client.software_specifications.list()

## Save and Deploy the model

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
MODEL_NAME = 'UNIVERSITY_ADMISSION_PREDICTION'
DEPLOYMENT_NAME = 'UNIVERSITY_ADMISSION_PREDICTION'
DEMO_MODEL = model

In [ ]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [ ]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [ ]:
#Save model
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [ ]:
model_details

In [ ]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

In [ ]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)